In [107]:
import torch
import random

from itertools import chain
from torch.autograd import Variable

import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [108]:
if torch.cuda.is_available():
    import torch.cuda as device
else:
    import torch as device

In [109]:
class Merger(nn.Module):
    def __init__(self, latent_factors_count, hidden_size):
        super(Merger, self).__init__()
        self.user_model = nn.Linear(latent_factors_count, hidden_size, bias=False)
        self.item_model = nn.Linear(latent_factors_count, hidden_size, bias=False)
        self.bias = nn.Parameter(torch.Tensor(1))
        
    def forward(self, user_embed, item_embed):
        return nn.Sigmoid()(self.user_model(user_embed) + self.item_model(item_embed) + self.bias)

In [110]:
class ContextMerger(nn.Module):
    def __init__(self, latent_factors_count, vocabulary_size, context_size):
        super(ContextMerger, self).__init__()
        self.user_model = nn.Linear(latent_factors_count, context_size, bias=False)
        self.item_model = nn.Linear(latent_factors_count, context_size, bias=False)
        self.rating_weight = nn.Parameter(torch.Tensor(1))
        self.review_model = nn.Linear(vocabulary_size, context_size, bias=False)
        self.bias = nn.Parameter(torch.Tensor(1))
        
    def forward(self, user_embed, item_embed, rating, review):
        return nn.Tanh()(
            self.user_model(user_embed) + self.item_model(item_embed) + \
            self.rating_weight * rating + self.review_model(review) + self.bias
        )

In [127]:
class EncoderModel(nn.Module):
    def __init__(self, users_count, items_count, latent_factors_count, vocabulary_size=333,
                 context_size=50, hidden_size=400, n_regression_layers=3, n_review_layers=1):
        super(EncoderModel, self).__init__()
        self.latent_factors_count = latent_factors_count

        self.user_embedding = nn.Embedding(users_count, latent_factors_count)
        self.item_embedding = nn.Embedding(items_count, latent_factors_count)
        
        self.merger = Merger(latent_factors_count, hidden_size)
        self.regression_model = nn.Sequential(
              *(list(chain.from_iterable([
                  [nn.Linear(hidden_size, hidden_size), nn.Sigmoid()]
                  for _ in range(n_regression_layers - 1)])) + \
              [nn.Linear(hidden_size, hidden_size), nn.Linear(hidden_size, 1)])
        )
        self.review_model = nn.Sequential(
            *(list(chain.from_iterable([
                      [nn.Linear(hidden_size, hidden_size), nn.Sigmoid()]
                      for _ in range(n_review_layers - 1)])) + \
                  [nn.Linear(hidden_size, vocabulary_size)])
        )
        self.context_merger = ContextMerger(latent_factors_count, vocabulary_size, context_size)

    def forward(self, input_user, input_item):
        embedded_user = self.user_embedding(input_user)
        embedded_item = self.item_embedding(input_item)
        
        merged = self.merger(embedded_user, embedded_item)
        regression_result = self.regression_model(merged)
        review_result = self.review_model(merged)
        review_softmax = nn.Softmax()(review_result)
        
        context = self.context_merger(embedded_user, embedded_item, regression_result, review_result)
        return regression_result, review_softmax, context

In [128]:
model = EncoderModel(10, 10, 20)

In [129]:
a, b, c = model.forward(Variable(torch.LongTensor([1, 2])),
                        Variable(torch.LongTensor([3, 4])))

In [139]:
class DecoderModel(nn.Module):
    def __init__(self, hidden_size=400, context_size=50, vocabulary_size=333):
        super(DecoderModel, self).__init__()
        
        self.embedding = nn.Embedding(vocabulary_size, context_size)
        self.gru = nn.GRU(context_size, context_size)
        self.out = nn.Linear(context_size, vocabulary_size)
        self.softmax = nn.LogSoftmax()
        
    def forward(self, input, hidden):
        output = self.embedding(input).view(len(input), 1, -1)
        hidden = hidden.view(hidden.size()[0], 1, -1)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output))
        return output, hidden

In [140]:
decoder = DecoderModel()

In [155]:
out, hidden = decoder.forward(Variable(torch.LongTensor([1, 2, 1, 1 ,2 , 2])), c)

In [157]:
torch.exp(out)

Variable containing:
( 0 ,.,.) = 
  0.1467  0.1619  0.1594  ...   0.1515  0.2121  0.1823

( 1 ,.,.) = 
  0.1800  0.1501  0.1731  ...   0.1638  0.1671  0.1857

( 2 ,.,.) = 
  0.1508  0.1657  0.1679  ...   0.1679  0.1702  0.1560

( 3 ,.,.) = 
  0.1428  0.1812  0.1622  ...   0.1682  0.1747  0.1449

( 4 ,.,.) = 
  0.1803  0.1744  0.1673  ...   0.1721  0.1442  0.1642

( 5 ,.,.) = 
  0.1994  0.1666  0.1701  ...   0.1764  0.1317  0.1670
[torch.FloatTensor of size 6x1x333]